In [11]:
%run StudyHelperFile.ipynb


Variables: my_trade
           aapl_put_df
           date_string_list
           date_datetime_list
           complete_c
           my_verticle
           
Function: add_days_to_date



`my_verticle` is a list of df of our trade that contains different strikes and it is used to determined our verticle width

In [12]:
my_verticle[0]

,QUOTE_DATE,EXPIRE_DATE,LAST_PRICE,DTE,STRIKE,P_ASK,P_BID,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_VOLUME,P_IV,PROB_ITM
0,2017-01-03,2017-02-10,116.14,38.0,112.0,2.05,1.94,-0.32230,0.03916,0.13607,-0.04561,97.0,0.24077,32.230
1,2017-01-03,2017-02-10,116.14,38.0,111.0,1.75,1.65,-0.28449,0.03675,0.12845,-0.04334,61.0,0.24231,28.449
2,2017-01-03,2017-02-10,116.14,38.0,110.0,1.49,1.41,-0.24884,0.03394,0.12039,-0.04061,48.0,0.24506,24.884
3,2017-01-03,2017-02-10,116.14,38.0,109.0,1.27,1.20,-0.21799,0.03104,0.11125,-0.03750,20.0,0.24941,21.799
4,2017-01-03,2017-02-10,116.14,38.0,108.0,1.07,0.99,-0.18822,0.02810,0.10236,-0.03536,65.0,0.25194,18.822
5,2017-01-03,2017-02-10,116.14,38.0,107.0,0.91,0.83,-0.16250,0.02524,0.09312,-0.03186,9.0,0.25620,16.250
6,2017-01-03,2017-02-10,116.14,38.0,106.0,0.81,0.69,-0.14163,0.02259,0.08526,-0.03036,2.0,0.26199,14.163
7,2017-01-03,2017-02-10,116.14,38.0,105.0,0.67,0.58,-0.12211,0.01995,0.07676,-0.02731,12.0,0.26572,12.211
8,2017-01-03,2017-02-10,116.14,38.0,104.0,0.59,0.51,-0.10608,0.01767,0.06975,-0.02541,82.0,0.27267,10.608
9,2017-01-03,2017-02-10,116.14,38.0,103.0,0.50,0.40,-0.09003,0.01541,0.06151,-0.02268,8.0,0.27576,9.003


In [13]:
made_trade_verticle = pd.DataFrame(columns = ['SOLD_STRIKE','BOUGHT_STRIKE','TRADE_DATE','TRADE_LAST_PRICE','DTE','EXIT_DATE','EXIT_LAST_PRICE','EXPIRE_DATE','AMOUNT_REC','AMOUNT_RISKED', 'AMOUNT_BUYBACK','PROFIT', 'PROFIT_PERCENT'])#change column since this is verticle trade
optimized_spread_list = []

# This has same index quote date as my_trade
verticle_width_compare = []
exit_date_list = []
entry_date_list = []

# this is needed to pass the very first loop and next_closest date value will determine after the loop under the if statement
next_closest_trade = my_trade['QUOTE_DATE'][0]

for trade in range(len(my_verticle)):
    spread = my_verticle[trade]
    # if we do not have enough spread data to find the width, we skip that spread df
    if len(spread) == 1:
        continue
    if ((spread['STRIKE'][0]) - (spread['STRIKE'].tail(1).tolist()[0])) < 5:
        continue
    optimized_spread = pd.DataFrame(columns = ['QUOTE_DATE','DTE','EXPIRE_DATE','STRIKE','WIDTH','NET_CREDIT','BUYING_POWER','POP_%','ROC','MARGINAL_COST'])
    # creates a dataframe that gives optimal trade width
    for i in range(1, len(spread)):
        width =  spread['STRIKE'][0] - spread['STRIKE'][i]
        credit = ((spread['P_ASK'][0] + spread['P_BID'][0]) / 2).round(2)
        bought = ((spread['P_ASK'][i] + spread['P_BID'][i]) / 2).round(2)   
        net_credit = credit - bought
        buying_power = (width - net_credit).round(2)
        roc = (net_credit / buying_power).round(3)
        pop = ((((net_credit / width) * 100) - 100)* -1).round(3)
        
        narrow_width =  spread['STRIKE'][0] - spread['STRIKE'][i-1]
        narrow_bought = ((spread['P_ASK'][i-1] + spread['P_BID'][i-1]) / 2).round(2)
        narrow_net_credit = credit - narrow_bought
        narrow_buying_power = (narrow_width - narrow_net_credit).round(2)
        if (narrow_buying_power != 0) & (narrow_net_credit != 0):
            narrow_roc = (narrow_net_credit / narrow_buying_power).round(3)
            marginal_cost = roc / narrow_roc
        else:
            marginal_cost = 0
        
        single_optimized_spread = pd.DataFrame([{'QUOTE_DATE' : spread['QUOTE_DATE'][i],
                                                    'DTE' : spread['DTE'][i],
                                                    'EXPIRE_DATE' : spread['EXPIRE_DATE'][i],
                                                    'STRIKE' : spread['STRIKE'][i],
                                                    'WIDTH' : width,
                                                    'NET_CREDIT' : net_credit,
                                                    'BUYING_POWER' : buying_power,
                                                    'ROC' : roc,
                                                    'POP_%' : pop,
                                                    'MARGINAL_COST' : marginal_cost}])
        optimized_spread = pd.concat([optimized_spread, single_optimized_spread])
        optimized_spread = optimized_spread.reset_index(drop = True)
    optimized_spread_list.append(optimized_spread)
    
    # filters the data between width 5-10 and finds the highest margin cost value
    highest_margin_cost = max(optimized_spread.loc[optimized_spread['WIDTH'].between(5,10)]['MARGINAL_COST'])
    highest_margin_strike = optimized_spread.loc[optimized_spread['MARGINAL_COST'] == highest_margin_cost]['STRIKE'].tolist()[0]
    
    # the very first row of our spread is where we are selling our option
    sold = spread.iloc[[0]]
    latest_expire_sold = sold['EXPIRE_DATE'].tolist()[0]
    latest_strike_sold = sold['STRIKE'].tolist()[0]
    latest_dte_sold = sold['DTE'].tolist()[0]
    sold_df = aapl_put_df.loc[(aapl_put_df['EXPIRE_DATE'] == latest_expire_sold) & (aapl_put_df['STRIKE'] == latest_strike_sold) & (aapl_put_df['DTE'] <= latest_dte_sold)]
    sold_df = sold_df.reset_index(drop = True)

    #sold_df and bought_df will ALWAYS have same (quote_date), (expire_date), (last_price), (dte)
    #because they are considered same trade when trading verticle
    
    # the highest margin strike we determined is where we are buying our option
    bought = spread.loc[spread['STRIKE'] == highest_margin_strike]
    latest_expire_bought = bought['EXPIRE_DATE'].tolist()[0]
    latest_strike_bought = bought['STRIKE'].tolist()[0]
    latest_dte_bought = bought['DTE'].tolist()[0]
    bought_df = aapl_put_df.loc[(aapl_put_df['EXPIRE_DATE'] == latest_expire_bought) & (aapl_put_df['STRIKE'] == latest_strike_bought) & (aapl_put_df['DTE'] <= latest_dte_bought)]
    bought_df = bought_df.reset_index(drop = True)
    
    spread_width = (sold_df['STRIKE'][0]) - bought_df['STRIKE'][0] #for dispaying data in df
    
    if (next_closest_trade == sold_df['QUOTE_DATE'][0]):
        buy_mid_list = []
        sell_mid_list = []
        win_percent = []
        profit_list = []
        buy_back = []
        # loops through the sold_df and lists all the data we need and we pick the data we need to store it 
        # in df by using the first true index
        for row in range(len(sold_df)):
            buy_ask = bought_df['P_ASK'][row]
            buy_bid = bought_df['P_BID'][row]

            sell_ask = sold_df['P_ASK'][row]
            sell_bid = sold_df['P_BID'][row]

            buy_mid = ((buy_ask + buy_bid) / 2).round(2)
            sell_mid = ((sell_ask + sell_bid) / 2).round(2)
            
            buy_mid_list.append(buy_mid)
            sell_mid_list.append(sell_mid)
            
            initial_credit = (sell_mid_list[0] - buy_mid_list[0]).round(2)
            exit_buy_back = (sell_mid - buy_mid).round(2)
            
            if exit_buy_back < 0:
                exit_buy_back = 0
            
            if exit_buy_back > spread_width:
                exit_buy_back = spread_width
            
            profit = (initial_credit - exit_buy_back).round(2)
            profit_percentage = (profit / initial_credit) * 100

            buy_back.append(exit_buy_back)
            win_percent.append(profit_percentage)
            profit_list.append(profit)
            
        boolean_list = [True if x >= 50 else False for x in win_percent]
        if True in boolean_list:
            # first_true_index is used to find the data we need from the list created above to input values into our output df
            first_true_index = boolean_list.index(True)
                
            exit_date = datetime.strptime(sold_df['QUOTE_DATE'][first_true_index], '%Y-%m-%d')
            if exit_date > date_datetime_list[-1]:
                break
            next_closest_trade = min([y for y in date_datetime_list if y >= exit_date], key=lambda j:abs(j-exit_date))
            next_closest_trade = next_closest_trade.strftime('%Y-%m-%d')
                
            single_made_trade_verticle = pd.DataFrame([{'SOLD_STRIKE' : sold_df['STRIKE'][0],
                                                                  'BOUGHT_STRIKE' : bought_df['STRIKE'][0],
                                                                  'TRADE_DATE' : sold_df['QUOTE_DATE'][0],
                                                                  'TRADE_LAST_PRICE': sold_df['LAST_PRICE'][0],
                                                                  'EXIT_DATE': sold_df['QUOTE_DATE'][first_true_index],
                                                                  'EXIT_LAST_PRICE': sold_df['LAST_PRICE'][first_true_index],
                                                                  'EXPIRE_DATE' : sold_df['EXPIRE_DATE'][0], 
                                                                  'DTE' : sold_df['DTE'][0], 
                                                                  'AMOUNT_REC' : initial_credit, 
                                                                  'AMOUNT_RISKED' : spread_width,
                                                                  'AMOUNT_BUYBACK' : buy_back[first_true_index], 
                                                                  'PROFIT': profit_list[first_true_index],
                                                                  'PROFIT_PERCENT' : win_percent[first_true_index]}])  
            made_trade_verticle = pd.concat([made_trade_verticle, single_made_trade_verticle])
            made_trade_verticle = made_trade_verticle.reset_index(drop=True)
        else:
            current_date = sold_df['QUOTE_DATE'][0]
            dte = sold_df['DTE'][0]
            exit_date = add_days_to_date(current_date, dte)  
            if exit_date > date_datetime_list[-1]:
                break
                    
            next_closest_trade = min([ y for y in date_datetime_list if y >= exit_date], key=lambda j:abs(j-exit_date))
            next_closest_trade = next_closest_trade.strftime('%Y-%m-%d')
               
            single_made_trade_verticle = pd.DataFrame([{'SOLD_STRIKE' : sold_df['STRIKE'][0],
                                                                  'BOUGHT_STRIKE' : bought_df['STRIKE'][0],
                                                                  'TRADE_DATE' : sold_df['QUOTE_DATE'][0],
                                                                  'TRADE_LAST_PRICE': sold_df['LAST_PRICE'][0],
                                                                  'EXIT_DATE': sold_df['QUOTE_DATE'].tail(1).tolist()[0],
                                                                  'EXIT_LAST_PRICE': sold_df['LAST_PRICE'].tail(1).tolist()[0],
                                                                  'EXPIRE_DATE' : sold_df['EXPIRE_DATE'][0], 
                                                                  'DTE' : sold_df['DTE'][0], 
                                                                  'AMOUNT_REC' : initial_credit, 
                                                                  'AMOUNT_RISKED' : spread_width,
                                                                  'AMOUNT_BUYBACK' : exit_buy_back, 
                                                                  'PROFIT': profit,
                                                                  'PROFIT_PERCENT' : profit_percentage}])
            made_trade_verticle = pd.concat([made_trade_verticle, single_made_trade_verticle])
            made_trade_verticle = made_trade_verticle.reset_index(drop=True)
                
made_trade_verticle

,SOLD_STRIKE,BOUGHT_STRIKE,TRADE_DATE,TRADE_LAST_PRICE,DTE,EXIT_DATE,EXIT_LAST_PRICE,EXPIRE_DATE,AMOUNT_REC,AMOUNT_RISKED,AMOUNT_BUYBACK,PROFIT,PROFIT_PERCENT
0,112.00,103.0,2017-01-03,116.14,38.00,2017-01-10,119.11,2017-02-10,1.55,9.00,0.73,0.82,52.903226
1,115.00,105.0,2017-01-10,119.11,45.00,2017-01-25,121.92,2017-02-24,1.58,10.00,0.63,0.95,60.126582
2,118.00,111.0,2017-01-26,122.00,43.00,2017-02-01,128.75,2017-03-10,1.20,7.00,0.19,1.01,84.166667
3,125.00,115.0,2017-02-01,128.75,43.96,2017-02-07,131.51,2017-03-17,1.31,10.00,0.53,0.78,59.541985
4,128.00,118.0,2017-02-07,131.51,44.96,2017-02-14,134.93,2017-03-24,1.28,10.00,0.52,0.76,59.375000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,330.00,320.0,2020-06-09,343.99,45.00,2020-06-22,358.90,2020-07-24,3.17,10.00,1.34,1.83,57.728707
64,347.50,337.5,2020-06-25,364.88,43.00,2020-07-10,383.68,2020-08-07,2.90,10.00,1.39,1.51,52.068966
65,365.00,357.5,2020-07-14,388.21,45.00,2020-07-31,425.50,2020-08-28,2.33,7.50,0.35,1.98,84.978541
66,126.25,120.0,2020-09-01,134.22,45.00,2020-10-16,118.96,2020-10-16,2.03,6.25,6.18,-4.15,-204.433498


In [6]:
verticle_put_sell_50_study = made_trade_verticle.copy()

In [7]:
verticle_put_sell_50_study.to_csv('verticle_put_sell_50_study.csv',index=False)